In [ ]:
from arena_api.system import system
from ultralytics import YOLO
import numpy as np
import cv2
import os
from datetime import datetime

# === Parameters ===
zoom_factor = 0.6
flip_180 = True
rotate_angle = 0  # Choose: 0, 90, 180, 270

# === Load YOLOv8 segmentation model ===
model = YOLO("yolov8n-seg.pt")  # Replace with your model path
print(f"✅ YOLOv8 model loaded, task: {model.task}")

# === Connect to Lucid camera ===
device_infos = system.device_infos
if not device_infos:
    raise Exception("❌ No Lucid cameras found.")
device = system.create_device(device_infos)[0]
print(f"✅ Connected to: {device.nodemap['DeviceModelName'].value}")

# === Set Pixel Format to BayerRG8 ===
try:
    device.nodemap.get_node("PixelFormat").value = "BayerRG8"
    print("✅ PixelFormat set to BayerRG8")
except Exception as e:
    print(f"❌ Failed to set PixelFormat: {e}")

# === Enable Auto Exposure, Gain, White Balance ===
try:
    device.nodemap.get_node("ExposureAuto").value = "Continuous"
    device.nodemap.get_node("GainAuto").value = "Continuous"
    device.nodemap.get_node("BalanceWhiteAuto").value = "Continuous"
    print("✅ Auto Exposure, Gain, and White Balance enabled")
except Exception as e:
    print(f"❌ Failed to enable auto settings: {e}")

# === Output setup ===
os.makedirs("output_frames", exist_ok=True)
output_video_path = "segmented_output_custom_capture.mp4"
video_writer = None

print("🎥 Real-time segmentation started. Press 'q' to quit.")

# === Standard output frame size ===
STANDARD_WIDTH = 640
STANDARD_HEIGHT = 480

# === Start Streaming and Inference ===
try:
    with device.start_stream():
        while True:
            buffer = device.get_buffer()

            # Convert Bayer buffer to 2D array
            img_bayer = np.array(buffer.data, dtype=np.uint8).reshape((buffer.height, buffer.width))
            device.requeue_buffer(buffer)

            # Convert Bayer to BGR
            img_bgr = cv2.cvtColor(img_bayer, cv2.COLOR_BAYER_RG2BGR)

            # === Digital Zoom ===
            h, w = img_bgr.shape[:2]
            cx, cy = w // 2, h // 2
            zw, zh = int(w * zoom_factor), int(h * zoom_factor)
            x1, x2 = max(cx - zw // 2, 0), min(cx + zw // 2, w)
            y1, y2 = max(cy - zh // 2, 0), min(cy + zh // 2, h)
            cropped = img_bgr[y1:y2, x1:x2]
            zoomed_img = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

            # === Flip and Rotation ===
            if flip_180:
                zoomed_img = cv2.flip(zoomed_img, -1)
            if rotate_angle == 90:
                zoomed_img = cv2.rotate(zoomed_img, cv2.ROTATE_90_CLOCKWISE)
            elif rotate_angle == 180:
                zoomed_img = cv2.rotate(zoomed_img, cv2.ROTATE_180)
            elif rotate_angle == 270:
                zoomed_img = cv2.rotate(zoomed_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            # === Convert to Grayscale ===
            gray_img = cv2.cvtColor(zoomed_img, cv2.COLOR_BGR2GRAY)

            # === Resize to standard resolution (640x480) ===
            gray_resized = cv2.resize(gray_img, (STANDARD_WIDTH, STANDARD_HEIGHT), interpolation=cv2.INTER_AREA)

            # === Convert grayscale back to BGR for YOLO input (YOLO expects 3 channels) ===
            input_img = cv2.cvtColor(gray_resized, cv2.COLOR_GRAY2BGR)

            # === Run YOLOv8 segmentation ===
            results = model.predict(input_img, verbose=False)[0]
            annotated_frame = results.plot()

            # === Display and Save ===
            cv2.imshow("YOLOv8 Segmentation", annotated_frame)

            # Initialize video writer if needed
            if video_writer is None:
                h_out, w_out = annotated_frame.shape[:2]
                video_writer = cv2.VideoWriter(
                    output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, (w_out, h_out)
                )
            video_writer.write(annotated_frame)

            # Save frame
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            cv2.imwrite(f"output_frames/frame_{timestamp}.jpg", annotated_frame)

            # Exit on keypress
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

except KeyboardInterrupt:
    print("🛑 Interrupted by user.")

finally:
    device.stop_stream()
    system.destroy_device(device)
    if video_writer:
        video_writer.release()
    cv2.destroyAllWindows()
    print("✅ Resources cleaned up.")
